In [1]:
import json
import pandas as pd

In [2]:
train_path = 'SCOTUS/train.json'
dev_path = 'SCOTUS/dev.json'

In [3]:
def load_data(path):
    with open(path, 'r') as f:
        data = json.load(f)
    return data

In [4]:
data = load_data(train_path)

In [7]:
html_content = data[1]["raw_source"]